
# ETfinder2 · Colab 笔记本（表单→聚类→树构建 一键跑）

> 这个笔记本把你当前的 `main.py` 管线搬到 Colab 运行。你只需要上传项目的 Python 模块和输入 JSON，就能在云端跑出 `clusters.json / index.tsv / FASTA / SSAP树 / Taxon树`。  
> **提示**：本笔记本不会修改你的源代码，只是提供一个标准化的运行环境。

## 1. 安装依赖（一次即可）

In [ ]:

# 安装必要的系统软件（安静模式）
!apt-get update -qq
!apt-get install -y -qq cd-hit mafft iqtree
# IQ-TREE 在某些镜像里叫 iqtree（不是 iqtree3），这里做个软链接，保证 main.py 默认参数可用
import shutil, os, subprocess, sys
def ensure_iqtree3():
    from shutil import which
    iq3 = which("iqtree3")
    if iq3:
        print("Found iqtree3:", iq3)
        return
    iq = which("iqtree")
    if iq:
        print("Found iqtree:", iq, "→ 创建 iqtree3 软链接")
        try:
            os.symlink(iq, "/usr/local/bin/iqtree3")
        except FileExistsError:
            pass
        print("Now iqtree3 ->", shutil.which("iqtree3"))
    else:
        print("没有找到 iqtree/iqtree3，请检查 apt 安装日志。")

ensure_iqtree3()

# Python 依赖（按你的工程需要可增删）
!pip -q install ete3 biopython
print("Deps installed.")

## 2. 准备工作目录并上传项目文件

In [ ]:

# 建立一个干净的工作目录结构
import os, pathlib, shutil
from pathlib import Path

WORK = Path("/content/ETfinder2")  # Colab 默认工作区
for sub in ["input","output","output/trees/ssap","output/trees/taxon","output/fasta","output/tmp","output/cdhit"]:
    (WORK / sub).mkdir(parents=True, exist_ok=True)

print("工作目录：", WORK)
print("请准备这些关键文件（通常在你的项目里都已存在）：")
print("  - main.py（你提供的主程序）")
print("  - 其依赖模块：M1_ssb2ssap.py, M2_taxontree.py, M3_ssaptree.py, M4_ssaptree.py, M5_ete.py, M6_exssapctx.py, sp2taxid.py（可选）, paths.py 等")
print("  - 输入 JSON：例如 input/scored_ctx_ctx0.json")

# 上传文件（可一次性多选）
from google.colab import files
uploaded = files.upload()

# 将上传的文件放入工作目录（保持原始文件名）
for fname in uploaded.keys():
    src = Path(fname)
    dst = WORK / src.name
    shutil.move(str(src), str(dst))
    print("已放置：", dst)

print("\n当前工作目录文件：")
!ls -lah /content/ETfinder2

## 3. 放置输入 JSON（如果还没放到 `/content/ETfinder2/input/`）

In [ ]:

# 如果你的输入 JSON 还没在 input/ 下，这里再上传一次并移动过去。
from google.colab import files
print("如果已经有 input/scored_ctx_ctx0.json，可以跳过执行本单元。")
more = files.upload()
from pathlib import Path
import shutil

for fname in more.keys():
    src = Path(fname)
    dst = Path("/content/ETfinder2/input") / src.name
    shutil.move(str(src), str(dst))
    print("已放置：", dst)

print("\ninput/ 目录内容：")
!ls -lah /content/ETfinder2/input || true

## 4. 运行管线（可在这里改参数）

In [ ]:

import os, sys, subprocess, textwrap, json
from pathlib import Path

WORK = Path("/content/ETfinder2")
os.chdir(WORK)
print("CWD =", os.getcwd())

# 可编辑：命令行参数
suffix   = "FDDEIPF"          # SSB 尾巴（决定输出命名前缀）
mode     = "rep"              # 'rep' 或 'all'
target   = ""                 # 目标物种（可留空）
json_in  = "input/scored_ctx_ctx0.json"   # 你的输入 JSON 路径
iqtree_bin = "iqtree3"        # 兼容前面软链

cmd = [
    "python", "main.py",
    "-w", str(WORK),
    "-j", json_in,
    "-s", suffix,
    "-m", mode,
]
if target:
    cmd += ["-t", target]
cmd += ["--iqtree-bin", iqtree_bin]

print("运行命令：", " ".join(cmd))
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("=== STDOUT ===\n", proc.stdout[:40000])
print("\n=== STDERR ===\n", proc.stderr[:40000])

rc = proc.returncode
print("\nReturn code:", rc)
if rc != 0:
    print("⚠️ 运行返回非 0，请检查上方日志（尤其是依赖模块是否已上传、输入 JSON 路径是否正确、外部程序是否安装成功）。")

## 5. 输出预览与下载

In [ ]:

from pathlib import Path
from IPython.display import display, Markdown

out = Path("/content/ETfinder2/output")
paths = {
    "clusters_json": out / "json" / f"{'FDDEIPF'}_ssap_clusters.json",
    "index_tsv":     out / "index.tsv",
    "fasta_tags":    out / "fasta" / f"{'FDDEIPF'}_ssap_all_with_tags.fasta",
    "fasta_reid":    out / "fasta" / f"{'FDDEIPF'}_ssap_host_all_reindexed.fasta",
    "ssap_tree_label": out / "trees" / "ssap" / "protein_tree.rep-only.labeled.nwk",
    "taxon_tree_label": out / "trees" / "taxon" / "taxon_tree.labeled.nwk",
}
print("可能的输出路径（按默认命名；若你改了 suffix/mode，文件名会不同）：")
for k,v in paths.items():
    print(" -", k, "->", v)

def head(p: Path, n=20):
    try:
        print(f"\n== {p} ==")
        with open(p, "r") as f:
            for i, line in enumerate(f):
                if i>=n: break
                print(line.rstrip())
    except Exception as e:
        print(f"[未找到或无法读取] {p}  ({e})")

# 简单预览（若存在）
for k,v in paths.items():
    if v.exists():
        head(v, n=20)

# 如需打包所有产物下载：
!cd /content/ETfinder2 && zip -r -q outputs.zip output
print("\n已打包：/content/ETfinder2/outputs.zip（包含 output/ 全部产物）")

# 在 Colab 里触发下载（也可以左侧文件栏手动下载）
from google.colab import files
if Path("/content/ETfinder2/outputs.zip").exists():
    files.download("/content/ETfinder2/outputs.zip")

## 6. （可选）挂载 Google Drive 持久化保存

In [ ]:

# 如果想把结果保存到你的 Google Drive：
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /content/drive/MyDrive/ETfinder2_runs
!cp -r /content/ETfinder2/output /content/drive/MyDrive/ETfinder2_runs/
print("已复制 output/ 到 /content/drive/MyDrive/ETfinder2_runs/")